In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
providers = pd.read_csv('providers.csv')
beneficiary = pd.read_csv('beneficiary.csv')
inpatients = pd.read_csv('inpatients.csv')

df_temp = pd.merge(inpatients, providers, how = 'inner', left_on = 'PID', right_on = 'PID')
df = pd.merge(df_temp, beneficiary, how = 'inner', left_on = 'BID', right_on = 'BID')
df

,BID,CID,StartDt,EndDt,PID,AmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,Chronic_Depression,Chronic_Diabetes,Chronic_IschemicHeart,Chronic_Osteoporasis,Chronic_rheumatoidarthritis,Chronic_stroke,InpatientAnnualReimbursementAmt,InpatientAnnualDeductibleAmt,OutpatientAnnualReimbursementAmt,OutpatientAnnualDeductibleAmt
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,1,2,1,1,36000,3204,60,70
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,1,2,1,1,36000,3204,60,70
3,BENE17521,CLM34721,2009-01-20,2009-02-01,PRV55912,19000,PHY349293,PHY370861,PHY363291,2009-01-20,...,2,2,1,2,2,2,19000,1068,100,20
4,BENE21718,CLM72336,2009-10-17,2009-11-04,PRV55912,17000,PHY334706,PHY334706,NaN,2009-10-17,...,2,2,1,1,2,2,17000,1068,1050,540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40469,BENE152832,CLM49682,2009-05-03,2009-05-06,PRV54063,3000,PHY386294,NaN,NaN,2009-05-03,...,1,1,1,2,2,2,3000,1068,710,330
40470,BENE153373,CLM53102,2009-05-28,2009-06-02,PRV54932,2000,PHY343021,PHY325388,NaN,2009-05-28,...,1,1,1,2,2,2,27000,2136,510,50
40471,BENE154457,CLM35582,2009-01-26,2009-01-29,PRV51424,6000,PHY351890,NaN,NaN,2009-01-26,...,2,2,2,2,2,2,6000,1068,1190,620
40472,BENE156872,CLM67045,2009-09-06,2009-09-13,PRV51572,5000,PHY407588,PHY311768,PHY311768,2009-09-06,...,1,1,1,2,2,1,5000,1068,2670,360


In [14]:
def group_by_digits(element):
  if str(element)[0] == 'E':
    return 'E'
  elif str(element)[0] == 'V':
    return 'V'
  elif str(element) == 'NaN':
    return '0'
  elif str(element) == 'OTH':
    return '0'
  else:
    # Extracting the first three digits as a string
    return str(element)[:3]
  
def combine_group(row, separator = ', '):
  parts = [group_by_digits(row[col]) for col in row.index]
  unique_parts = [v for i, v in enumerate(parts) if i == 0 or v != parts[i - 1]]
  return separator.join(unique_parts)
  #return ', '.join([group_by_digits(row[col]) for col in row.index])

In [15]:
dia_col_list = ['DiagnosisGroupCode', 'DiagnosisCode_1', 'DiagnosisCode_2', 'DiagnosisCode_3', 'DiagnosisCode_4', 'DiagnosisCode_5', 'DiagnosisCode_6', 'DiagnosisCode_7', 'DiagnosisCode_8', 'DiagnosisCode_9', 'DiagnosisCode_10']
df['Combined_Dia'] = df[dia_col_list].apply(combine_group, axis=1)
df['Combined_Dia']

0        201, 197, 401, 585, 784, 276, 715, 272, 198, 5...
1                                  750, 618, 294, 564, nan
2                     883, 296, 303, 716, 345, V, 327, nan
3        987, 424, 263, 294, 403, 458, 284, 585, 275, E...
4        941, V, 416, 733, 781, 799, 788, 722, 438, 401...
                               ...                        
40469    076, 250, 272, 305, 782, 715, 424, 414, 278, 3...
40470                           727, 600, 536, 596, V, nan
40471                    391, 562, 272, 729, 287, 792, nan
40472    190, 486, 790, V, 599, 285, 790, 491, 414, 263...
40473       238, 410, 785, 585, V, 578, 398, 403, 286, nan
Name: Combined_Dia, Length: 40474, dtype: object

In [21]:
# dummies = pd.get_dummies(df['Combined_Dia'], sep)
dummies = df['Combined_Dia'].str.get_dummies(sep=', ')
dummies

,000,001,002,003,004,005,006,007,008,009,...,994,995,996,997,998,999,E,OTH,V,nan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
40470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
40471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
40472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [23]:
dummies['fraud'] = df['Fraud'].replace(['Yes', 'No'], [1, 0])
dummies

,000,001,002,003,004,005,006,007,008,009,...,995,996,997,998,999,E,OTH,V,nan,fraud
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
40470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
40471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
40472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
